# MobileNetV2 model 
## outline of the code

1. Import necessary libraries:
* Import TensorFlow, Keras, and other necessary libraries for building and training the model.

2. Load the dataset:
* Load the face mask dataset, which includes images of people with and without masks.

3. Data preprocessing:

* Resize the images to a consistent size (e.g., 224x224 pixels) and normalize the pixel values.
* Apply data augmentation techniques (e.g., rotation, flipping, zooming) to artificially increase the size and diversity of the dataset.
* Split the dataset into training and test sets for model evaluation.

4. Create a base model with transfer learning:

* Load a pre-trained model (MobileNetV2) as the base model.
* Remove the top layers to use the pre-trained model as a feature extractor.

5. Add regularization and custom layers:

* Apply regularization techniques (dropout) to reduce overfitting.
* Add custom layers on top of the base model to create a new model architecture tailored for the face mask classification task.

6. Compile the model:

* Compile the model with an optimizer (Adam), loss function (binary_crossentropy), and metric (accuracy).


7. Train the model:

* Train the model on the training dataset for a specified number of epochs, using the test dataset for validation.

8. Evaluate the model:

* Assess the model's performance on the test dataset and report the test accuracy.


In this repo, we implements a face mask detection model using transfer learning and data augmentation, with regularization techniques and a custom model architecture. The model is trained on a dataset of images of people with and without masks and evaluated on a separate test dataset to measure its performance.

In [77]:
import boto3
import tensorflow as tf
from PIL import Image
import numpy as np
from retinaface import RetinaFace
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.regularizers import l2


In [43]:
s3 = boto3.resource('s3')
bucket_name = 'dalle2images'
real_image_paths = []
fake_image_paths = []
real_labels = []
fake_labels = []

In [44]:
# loop through the images in your S3 bucket and append them to their respective lists:
for obj in s3.Bucket(bucket_name).objects.all():
    if obj.key.endswith('.png') and obj.key.startswith('real'):
        real_image_paths.append(obj.key)
        real_labels.append(0)
    elif obj.key.endswith('.png') and obj.key.startswith('fake'):
        fake_image_paths.append(obj.key)
        fake_labels.append(1)


In [70]:
# randomly select 1000 images (500 real, 500 fake) from your bucket
n_images = 100
n_real = n_fake = n_images // 2
real_samples = np.random.choice(real_image_paths, n_real, replace=False)
fake_samples = np.random.choice(fake_image_paths, n_fake, replace=False)
sample_paths = np.concatenate([real_samples, fake_samples])


In [71]:
images = []
labels = []
image_size = (224, 224)

for i, path in enumerate(sample_paths):
    img = s3.Object(bucket_name, path)
    img = Image.open(img.get()['Body'])
    img = img.resize(image_size, Image.ANTIALIAS)
    img = np.array(img)
    faces = RetinaFace.detect_faces(img)

    if len(faces) == 0:
        continue

    img = tf.cast(img, dtype=tf.float32)
    images.append(img)
    labels.append(0 if path.startswith('real') else 1)

images = np.stack(images)
labels = np.array(labels)


In [72]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

train_dataset = train_dataset.shuffle(buffer_size=len(train_images), reshuffle_each_iteration=True)

batch_size = 32
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)



In [78]:

# Load the pre-trained MobileNetV2 model without the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Add the data augmentation layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
])

# Define the regularization strength
reg_strength = 0.001

# Create a new model using the base model and adding custom top layers
model = tf.keras.models.Sequential([
    data_augmentation,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(reg_strength)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(reg_strength)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(reg_strength)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
history = model.fit(train_dataset, epochs=epochs, validation_data=test_dataset)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print("Test accuracy: {:.2f}%".format(accuracy * 100))

Epoch 1/10
3/3 [==============================] - 6s 747ms/step - loss: 1.9344 - accuracy: 0.4500 - val_loss: 1.3408 - val_accuracy: 0.5500
Epoch 2/10
3/3 [==============================] - 1s 297ms/step - loss: 1.6811 - accuracy: 0.5125 - val_loss: 1.3065 - val_accuracy: 0.6500
Epoch 3/10
3/3 [==============================] - 1s 298ms/step - loss: 1.5161 - accuracy: 0.5250 - val_loss: 1.3130 - val_accuracy: 0.6500
Epoch 4/10
3/3 [==============================] - 1s 311ms/step - loss: 1.4481 - accuracy: 0.5875 - val_loss: 1.3961 - val_accuracy: 0.6000
Epoch 5/10
3/3 [==============================] - 1s 306ms/step - loss: 1.4575 - accuracy: 0.5250 - val_loss: 1.3506 - val_accuracy: 0.6000
Epoch 6/10
3/3 [==============================] - 1s 325ms/step - loss: 1.3380 - accuracy: 0.6000 - val_loss: 1.2909 - val_accuracy: 0.6500
Epoch 7/10
3/3 [==============================] - 1s 334ms/step - loss: 1.2294 - accuracy: 0.6500 - val_loss: 1.2412 - val_accuracy: 0.6500
Epoch 8/10
3/3 [====